# Introduction
This notebook is a more detailed description of the algorithm we will implement in `c++` & `CUDA` for reference. It contains: 
- Two identic approaches to the algorithm:
    - [One](#basic-numpy) using classic `numpy` and
    - [the other](#cupy-implementation) using `cuPY`, the `numpy` counterpart for `CUDA`
- [Benchmarks](#benchmark) for both of them.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt

# Basic Numpy
We start with the `numpy` implementation, these are the steps we will follow:
- Generate the samples
- Build the estimator:
    - Define the ranges for the parameters: In the real world we would need to play a bit with the numbers by starting with a very wide range as we won't know where $\mu$ and $\sigma$ fall. But, since this is a contrived example, we can be a bit self-indulgent and use the information we defined to generate the random variates.
    - With these ranges we then build the outer product so we will have all the combinations for $\mu$, $\sigma$ & $x$ 
    - Build the prior: for the sake of simplicity we use a flat prior which won't have impact on the posterior. Had we had a reasonable assumption that the parameters are these or those, we should have set it as our prior.
    - Compute the likelihood: this step involves two operations:
        - First we compute the densities for each triplet of $\mu$, $\sigma$ & $x$, where $x$ are the observations.
        - Second, we reduce the densities to a single number by taking the product over the second axis, ie, the densities. This is because we can think of those densities as the joint probability of observation 1 and observation 2...observation $n$. This product can become really small, so if we were only interested in the point estimates, we could take the log-likelihood by using `np.log(densities).sum(axis=2)`. However, since we don't have too many observations and parameter pairs, we will stick to the conventional product.
    - Compute the posterior: we do the classic $\mathrm{prior}\times\mathrm{likelihood}/\mathrm{evidence}$, but notice that the product is not really necessary as we are using a flat prior, so a bare normalization will do the trick as well.
- Print the outcomes.

In [ ]:
mu, sigma = 20, 2  # the values we want to infer
np.random.seed(42)  # Let's fix the samples for reproduction purposes

# Generate and display some of the observations
observations_np = np.random.normal(mu, sigma, 50).astype(np.float32)
observations_np[:10]

In [ ]:
def numpy_estimator(observations):
    """
    Compute the posterior distribution of a set of observations.

    Arguments:
        - observations (np.ndarray): a numpy array containing the the observations

    Returns:
        - a tuple of np.ndarray:
            - posterior (np.ndarray[101]): the posterior probability of the observations.
            - mu_range (np.ndarray[101]): the range of parameters used for the mean.
            - sigma_range (np.ndarray[101]): the range of parameters used for the std.
    """
    # Define the ranges
    ranges_size = 101
    mu_range = np.linspace(18, 22, ranges_size, dtype=np.float32)
    sigma_range = np.linspace(1, 3, ranges_size, dtype=np.float32)

    # Build the outer product
    mu_grid, sigma_grid, observations_grid = np.meshgrid(
        mu_range, sigma_range, observations
    )

    # build the prior
    prior = np.ones((ranges_size, ranges_size))

    # compute the likelihood
    densities = norm(mu_grid, sigma_grid).pdf(observations_grid)
    likes = densities.prod(axis=2)
    # Alternatively we could use the log-likelihood, but it will spoil the marginals.
    # np.log(densities).sum(axis=2)

    # Compute posteriors.
    posterior = likes * prior  # this guy has no effect
    posterior /= posterior.sum()  # normalization

    # Return the expectations
    return posterior, mu_range, sigma_range

posterior, mu_range, sigma_range = numpy_estimator(observations_np)
print(
    np.sum(mu_range * posterior.sum(axis=1)),
    np.sum(sigma_range * posterior.sum(axis=0))
)

## Visualization
One of the nice things of doing Bayesian Statistics is that one does not get just a point estimate but rather a distribution that gives a feeling of how confident one should be about that point estimate.

In the following plots, we display this confidence as a:
- Posterior joint distribution among `mu_range` and `sigma_range`
- Marginals for `mu` and `sigma`

Computing these credible intervals is out of the scope of this project but it's something very straightforward to do by means of the CDF:

```python
def credible_interval_90(distribution):
    """Return the values that lay in the 90% of the distribution."""

    def find_quantile(p):
        """Get the index where the cdf overcomes some prob p."""
        cdf = distribution.cumsum()
        quantile = distribution[cdf > p][0]
        return np.where(distribution == quantile)[0][0]

    lo, hi = [find_quantile(p) for p in (.05, .95)]
    return distribution[lo:hi]
```

You can check [this source](https://allendowney.github.io/ThinkBayes2/chap05.html#credible-intervals) to know more about them.

In [ ]:
# Visualize
df = pd.DataFrame(posterior, index=sigma_range.round(2), columns=mu_range.round(2))
_, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 4))
ax1 = sns.heatmap(df, cmap="YlGnBu", ax=ax[0])
ax1.invert_yaxis()
ax1.set_title("Posterior Joint Distribution")
ax1.set_xlabel("mu")
ax1.set_ylabel("sigma")

ax[1].plot(mu_range, posterior.mean(axis=1))
ax[1].set_title("Mu Marginal")
ax[1].set_xlabel("$\mu$")
ax[1].set_ylabel("$p(\mu|data)$")

ax[2].plot(sigma_range, posterior.mean(axis=0))
ax[2].set_title("Sigma Marginal")
ax[2].set_xlabel("$\sigma$")
ax[2].set_ylabel("$p(\sigma|data)$");

# CuPy Implementation

This implementation is equivalent to the [numpy one](#basic-numpy) but using the `CuPy` [library](https://github.com/cupy/cupy)

The main difference with previous approach is that in `CuPy` we can't make use of the `scipy` library and therefore we need to reinvent the wheel a bit and create the pdf of a normal distribution that is happy to work with `CuPy`

In [ ]:
import cupy as cp

observations_cp = cp.asarray(observations_np, dtype=cp.float32)

def cp_normal(mu_grid, sigma_grid, observations_grid):
    """
    Compute the pdf of a normal distribution

    Arguments:
        - mu_grid (cp.ndarray): the space of parameters for the mean as a 101x101x50 array
        - sigma_grid (cp.ndarray): the space of parameters for the std as a 101x101x50 array
        - observations_grid (cp.ndarray): the space of observations as a 101x101x50 array

    Returns:
        - a 101x101x101 cp.ndarray with the densities per triplet mu, sigma & x
    """
    normalization = cp.sqrt(2 * cp.pi) * sigma_grid
    return cp.exp(-0.5 * ((observations_grid - mu_grid) / sigma_grid) ** 2) / normalization


def cupy_estimator(observations):
    """
    Compute the posterior distribution of a set of observations.

    Arguments:
        - observations (cp.ndarray): a CuPy array containing the the observations

    Returns:
        - a tuple of cp.ndarray:
            - posterior (cp.ndarray[101]): the posterior probability of the observations.
            - mu_range (cp.ndarray[101]): the range of parameters used for the mean.
            - sigma_range (cp.ndarray[101]): the range of parameters used for the std.
    """
    # Define ranges
    grid_size = 101
    mu_range = cp.linspace(18, 22, grid_size, dtype=cp.float32)
    sigma_range = cp.linspace(1, 3, grid_size, dtype=cp.float32)

    # Build the outer product of mu, sigma and observations.
    mu_grid, sigma_grid, observations_grid = cp.meshgrid(mu_range, sigma_range, observations)

    # Build the prior
    prior = cp.ones((grid_size, grid_size))

    # compute the likelihood. In this case we can't use the scipy norm as it only works with
    # numpy arrays, so we will need to craft our own.
    densities = cp_normal(mu_grid, sigma_grid, observations_grid)
    likes = densities.prod(axis=2)

    # Compute posterior
    posterior = likes * prior
    posterior /= posterior.sum()

    # Return the expectations
    return posterior, mu_range, sigma_range

posterior, mu_range, sigma_range = cupy_estimator(observations_cp)
print(
    cp.sum(mu_range * posterior.sum(axis=1)),
    cp.sum(sigma_range * posterior.sum(axis=0))
)

# Benchmark
As a final step we run some benchmarks to realise the improvement. Note that the numbers got here may differ from what you actually get in your setup. We can check that:
- Numpy: 31.2ms
- CuPY: .969ms (CPU) + 1.4ms (GPU) = 2.36ms

## Numpy

In [6]:
%timeit numpy_estimator(observations_np)


33 ms ± 4.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## CuPY

In [7]:
from cupyx.profiler import benchmark
benchmark(cupy_estimator, (observations_cp, ), n_repeat=20)

cupy_estimator      :    CPU:   767.068 us   +/- 84.480 (min:   630.017 / max:   909.773) us     GPU-0:  1241.259 us   +/- 232.582 (min:   985.696 / max:  1724.416) us